# Agentive Chatbot

In [49]:
import os
import time
import datetime
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display
from dotenv import load_dotenv
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder
from semantic_router.layer import RouteLayer
from openai import OpenAI
import yaml
from enum import Enum

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")
encoder = OpenAIEncoder()

def load_routes(agent_dir):
    routes = []
    prompts = {}
    for filename in os.listdir(agent_dir):
        if filename.endswith(".yaml"):
            filepath = os.path.join(agent_dir, filename)
            with open(filepath, 'r') as file:
                agent_data = yaml.safe_load(file)
            if 'utterances' in agent_data:
                route = Route(
                    name=agent_data['route_name'],
                    utterances=agent_data['utterances']
                )
                routes.append(route)
            if 'prompt' in agent_data:
                prompts[agent_data['agent_name']] = agent_data['prompt']
    return routes, prompts

emo_routes, emo_prompts = load_routes("base_emotional_states")
angry_routes, angry_prompts = load_routes("angry_emotion")
anxiety_routes, anxiety_prompts = load_routes("anxiety_emotion")
excited_routes, excited_prompts = load_routes("excited_emotion")
knowledge_routes, knowledge_prompts = load_routes("knowledge")

#routers
base_emotion_rl = RouteLayer(encoder = encoder, routes = emo_routes)
angry_emotion_rl = RouteLayer(encoder = encoder, routes = angry_routes)
anxiety_emotion_rl = RouteLayer(encoder = encoder, routes = anxiety_routes)
excited_emotion_rl = RouteLayer(encoder = encoder, routes = excited_routes)
knowledge_rl = RouteLayer(encoder = encoder, routes = knowledge_routes)

# memory_rl = RouteLayer(encoder = encoder, routes = memory_routes)
# attitude_rl = RouteLayer(encoder = encoder, routes = attitude_routes)

#### Variables to store the default system prompt and scenario prompt

In [50]:
with open('syst_prompt.txt', 'r') as f:
    system_prompt = f.read()

with open('scenario_prompt.txt', 'r') as f:
    scenario_prompt = f.read()

#### Defines logic for updating, activating, and deactivating prompts

In [51]:
class EmotionalState(Enum):
    OK = "ok"
    ANGER = "anger"
    ANXIETY = "anxiety"
    EXCITED = "excited"

class ANGRYSTATES(Enum):
    OK = 0
    ANNOYED = 1
    FRUSTRATED = 2
    ANGRY = 3
    ENRAGED = 4

class ANXIETYSTATES(Enum):
    OK = 0
    UNEASY = 1
    CONCERNED = 2
    WORRIED = 3
    ANXIOUS = 4

class EXCITEDSTATES(Enum):
    OK = 0
    INTERESTED = 1
    ENGAGED = 2
    THRILLED = 3
    EXHILARATED = 4

CURRENT_BASE_STATE = EmotionalState.OK
CURRENT_ANGRY_SUB_STATE = ANGRYSTATES.OK
CURRENT_ANXIETY_SUB_STATE = ANXIETYSTATES.OK
CURRENT_EXCITED_SUB_STATE = EXCITEDSTATES.OK

def establish_base_state(prompt: str) -> None:
    """ 
    Establishes the basic emotion 
    Should only be called when current base state is 'ok' 
    """
    global CURRENT_BASE_STATE
    r = base_emotion_rl(prompt)
    if r.name is not None:
        if r.name == "anger":
            CURRENT_BASE_STATE = EmotionalState.ANGER
        elif r.name == "anxiety":
            CURRENT_BASE_STATE = EmotionalState.ANXIETY
        elif r.name == "excited":
            CURRENT_BASE_STATE = EmotionalState.EXCITED
    else:
        CURRENT_BASE_STATE = EmotionalState.OK

def route_to_second_emotion(prompt: str) -> None:
    """ 
    Calls a function to update emotion based on the current emotion
    """
    global CURRENT_BASE_STATE
    global CURRENT_ANGRY_SUB_STATE
    global CURRENT_ANXIETY_SUB_STATE
    if CURRENT_BASE_STATE == EmotionalState.OK:
        establish_base_state(prompt)
    
    if CURRENT_BASE_STATE == EmotionalState.ANXIETY:
        prompt, question_category = update_anxiety_prompt(prompt)
    elif CURRENT_BASE_STATE == EmotionalState.ANGER:
        prompt, question_category = update_angry_prompt(prompt)
    elif CURRENT_BASE_STATE == EmotionalState.EXCITEd:
        prompt, question_category = update_excited_prompt(prompt)
    else: 
        prompt = emo_prompts['ok']
        question_category = 'ok'
    return prompt, question_category

def angry_state_to_prompt(state: ANGRYSTATES) -> str:
    state_mapping = {
        ANGRYSTATES.OK: "ok",
        ANGRYSTATES.ANNOYED: "annoyed",
        ANGRYSTATES.FRUSTRATED: "frustrated",
        ANGRYSTATES.ANGRY: "angry",
        ANGRYSTATES.ENRAGED: "enraged",
    }
    return state_mapping[state]

def classify_angry_emotion(prompt: str) -> str:
    r = angry_emotion_rl(prompt)
    return r.name

def update_angry_prompt(user_input: str) -> str:
    global CURRENT_ANGRY_SUB_STATE
    global CURRENT_BASE_STATE
    question_category = classify_angry_emotion(user_input)
    print(f"emotional sub route: {question_category}")
    prompt = ""
    if question_category == 'escalate':
        if CURRENT_ANGRY_SUB_STATE != ANGRYSTATES.ENRAGED:
            CURRENT_ANGRY_SUB_STATE = ANGRYSTATES(CURRENT_ANGRY_SUB_STATE.value + 1)
    elif question_category == 'de_escalate':
        if CURRENT_ANGRY_SUB_STATE != ANGRYSTATES.OK:
            CURRENT_ANGRY_SUB_STATE = ANGRYSTATES(CURRENT_ANGRY_SUB_STATE.value - 1)
    prompt_key = angry_state_to_prompt(CURRENT_ANGRY_SUB_STATE)
    prompt = angry_prompts[prompt_key]
    print(f"emotional sub prompt: {prompt}")
    if CURRENT_ANGRY_SUB_STATE == ANGRYSTATES.OK:
        CURRENT_BASE_STATE = EmotionalState.OK
    return prompt, question_category

def anxiety_state_to_prompt(state: ANXIETYSTATES) -> str:
    state_mapping = {
        ANXIETYSTATES.OK: "ok",
        ANXIETYSTATES.UNEASY: "uneasy",
        ANXIETYSTATES.CONCERNED: "concerned",
        ANXIETYSTATES.WORRIED: "worried",
        ANXIETYSTATES.ANXIOUS: "anxious",
    }
    return state_mapping[state]

def classify_anxiety_emotion(prompt: str) -> str:
    r = anxiety_emotion_rl(prompt)
    return r.name

def update_anxiety_prompt(user_input: str) -> str:
    global CURRENT_ANXIETY_SUB_STATE
    global CURRENT_BASE_STATE
    question_category = classify_anxiety_emotion(user_input)
    print(f"emotional sub route: {question_category}")
    prompt = ""
    if question_category == 'escalate':
        if CURRENT_ANXIETY_SUB_STATE != ANXIETYSTATES.ANXIOUS:
            CURRENT_ANXIETY_SUB_STATE = ANXIETYSTATES(CURRENT_ANXIETY_SUB_STATE.value + 1)
    elif question_category == 'de_escalate':
        if CURRENT_ANXIETY_SUB_STATE != ANXIETYSTATES.OK:
            CURRENT_ANXIETY_SUB_STATE = ANXIETYSTATES(CURRENT_ANXIETY_SUB_STATE.value - 1)
    prompt_key = anxiety_state_to_prompt(CURRENT_ANXIETY_SUB_STATE)
    prompt = anxiety_prompts[prompt_key]
    print(f"emotional sub prompt: {prompt}")
    if CURRENT_ANXIETY_SUB_STATE == ANXIETYSTATES.OK:
        CURRENT_BASE_STATE = 'ok'
    return prompt, question_category

def excited_state_to_prompt(state: ANGRYSTATES) -> str:
    state_mapping = {
        EXCITEDSTATES.OK: "ok",
        EXCITEDSTATES.INTERESTED: "interested",
        EXCITEDSTATES.ENGAGED: "engaged",
        EXCITEDSTATES.THRILLED: "thrilled",
        EXCITEDSTATES.EXHILARATED: "exhilarated",
    }
    return state_mapping[state]

def classify_excited_emotion(prompt: str) -> str:
    r = excited_emotion_rl(prompt)
    return r.name

def update_excited_prompt(user_input: str) -> str:
    global CURRENT_EXCITED_SUB_STATE
    global CURRENT_BASE_STATE
    question_category = classify_excited_emotion(user_input)
    print(f"emotional sub route: {question_category}")
    prompt = ""
    if question_category == 'escalate':
        if CURRENT_EXCITED_SUB_STATE != EXCITEDSTATES.EXHLARATED:
            CURRENT_EXCITED_SUB_STATE = EXCITEDSTATES(CURRENT_EXCITED_SUB_STATE.value + 1)
    else:
        CURRENT_EXCITED_SUB_STATE = EXCITEDSTATES.OK
        CURRENT_BASE_STATE = EmotionalState.OK
    prompt_key = excited_state_to_prompt(CURRENT_EXCITED_SUB_STATE)
    prompt = excited_prompts[prompt_key]
    print(f"emotional sub prompt: {prompt}")
    return prompt, question_category

#returns one attitude or belief route

def classify_knowledge(prompt: str) -> str:
    r = knowledge_rl(prompt)
    return r.name

def update_knowledge_prompt(user_input: str) -> str:
    question_category = classify_knowledge(user_input)
    print(f"agent route: {question_category}")
    if question_category is None:
        prompt = "" #default prompt
        route = "No route"
    else:
        prompt = knowledge_prompts[question_category]
        route = question_category
    return prompt, route

In [52]:
class ChatBot():  
    def __init__(self, api_key, role): 
        self.client = OpenAI(api_key=api_key)
        self.role = role
        self.messages = [{"role": "user", "content": ""}]
        self.all_conversations = []
    
    def query(self, query: str, emo_prompt: str, emo_route: str, knowledge_prompt: str, knowledge_agent: str) -> None:
        self.messages.append({"role": "user", "content": query})
        self.messages.insert(0, {"role": "system", "content": self.role + scenario_prompt + emo_prompt + knowledge_prompt})
        try:
            stream = self.client.chat.completions.create(
                model="gpt-4o-2024-08-06", messages=self.messages,
                stream=True,
            )
            text = []
            for part in stream:
                if part.choices[0].delta.content is not None:
                    response_part = part.choices[0].delta.content
                    print(response_part, end="", flush=True)
                    text.append(response_part)
            full_reply_content = ''.join([m for m in text if m is not None])
            self.messages.append({"role": "assistant", "content": full_reply_content})
            self.messages.pop(0)
            self.all_conversations.append(f"\n Emotion route: {emo_route} \nEmotion prompt: {emo_prompt} \nKnowledge agent {knowledge_agent}")
            self.all_conversations.append(self.messages[-2:])
            print('\n')

        except Exception as e:
            print(f"An error occurred: {e}")

    def save_whole_conversation(self, filename):
        with open(f"transcripts/{filename}", "w") as file:
            for conversation in self.all_conversations:
                if isinstance(conversation, str):
                    file.write(conversation + "\n")
                elif isinstance(conversation, list):
                    for message in conversation:
                        if message["role"] == "user":
                            file.write("User: " + message["content"] + "\n")
                        elif message["role"] == "assistant":
                            file.write("Agent: " + message["content"] + "\n")

#initialises chatbot
rpa = ChatBot(api_key=api_key, role = system_prompt)

In [ ]:
def on_pasted_submit(b):
    pasted_content = pasted_input_field.value 
    EMO_PROMPT, EMO_ROUTE = route_to_second_emotion(pasted_content)
    print(f"Route: {CURRENT_BASE_STATE}")
    print(f"current emo prompt: {EMO_PROMPT}")
    KNOWLEDGE_PROMPT, KNOWLEDGE_ROUTE = update_knowledge_prompt(pasted_content)
    rpa.query(pasted_content, emo_prompt = EMO_PROMPT, emo_route = EMO_ROUTE, knowledge_prompt = KNOWLEDGE_PROMPT, knowledge_agent = KNOWLEDGE_ROUTE)
    time.sleep(1)
    pasted_input_field.value = ""

def on_save_click(b):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{timestamp}.txt"
    rpa.save_whole_conversation(filename)

pasted_input_field = widgets.Textarea(
    placeholder='Type here :)',
    layout=widgets.Layout(width='100%', height='50px')
)
pasted_submit_button = widgets.Button(description='Submit')
pasted_submit_button.style.button_color = 'lightpink'

save_button = widgets.Button(description='Save')
save_button.style.button_color = 'lightgrey'

pasted_submit_button.on_click(on_pasted_submit)
save_button.on_click(on_save_click)

button_layout = HBox([pasted_submit_button, save_button], layout=Layout(justify_content='center'))

display(pasted_input_field, button_layout)

Textarea(value='', layout=Layout(height='50px', width='100%'), placeholder='Type here :)')

Route: EmotionalState.OK
current emo prompt: Your main goals for this conversation is to find out more about the insurance policies that the insurance agent sells, and also ascertain if the agent is trustworthy to work with.
agent route: insurance
Sure, I'd be happy to explain! FWD Group is a pan-Asian life insurance business, with over 10 million customers across 10 markets. Their "Set for Life" plan is particularly noteworthy because it combines both protection benefits and investment opportunities. 

With "Set for Life," life protection is provided up to the age of 100. Upon the unfortunate event of the policyholder's death, the beneficiaries receive the higher of either 100% of the benefit amount, the account value, or the minimum death benefit. 

One unique aspect is the diverse investment funds available, which cater to different risk appetites. This means a portion of your premium is invested in funds you choose, with higher-risk equity funds potentially offering higher returns,